<a href="https://colab.research.google.com/github/OdysseusPolymetis/digital_classics_course/blob/main/5_scansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cltk

In [ ]:
from cltk.prosody.lat.hexameter_scanner import HexameterScanner

In [ ]:
scanner = HexameterScanner()

line = "arma virumque cano Troiae qui primus ab oris"

scansion = scanner.scan(line)
print(scansion.scansion)
print(scansion.accented)
print(scansion.scansion_notes)

Ce qui donne quelque chose comme ça:
<br>— ᴗ ᴗ   |  — ᴗ ᴗ  |  — —  ||  — —  |  — ᴗ ᴗ  |  — —
<br>arma vi | rumque ca | nō Trō- || jae quī | prīmu sa | bō rīs
<br>D      |    D      |   S     ||   S     |    D     |   S
<br>Vous pouvez trouver tous les types de vers que `cltk` est capable d'analyser en latin ici : [liste des mètres](https://v1.cltk.org/en/latest/cltk.prosody.lat.html)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from cltk.prosody.lat.hexameter_scanner import HexameterScanner

scanner = HexameterScanner()

with open("/content/votreTexte.txt", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        sc = scanner.scan(line)
        print(line)
        print("  ", sc.scansion)
        print("  ", sc.accented)
        print()

Pour le grec maintenant

In [ ]:
from cltk.prosody.grc import Scansion

scanner_grc = Scansion()

vers = "μῆνιν ἄειδε, θεὰ, Πηληϊάδεω Ἀχιλῆος."

In [ ]:
patterns = scanner_grc.scan_text(vers)
print(patterns)

Bref, globalement, en grec, ça marche BEAUCOUP moins bien. Pour le grec, vous pouvez tenter, mais avec un texte très court. Et on n'a pas les mètres explicites.

In [ ]:
with open("mon_fichier_grec.txt", encoding="utf-8") as f:
    for line in f:
        vers = line.strip()
        if not vers:
            continue

        if not vers.endswith("."):
            vers_for_scan = vers + "."
        else:
            vers_for_scan = vers

        patterns = scanner_grc.scan_text(vers_for_scan)

        if not patterns:
            print("Pas de pattern pour :", vers)
            continue

        pattern = patterns[0]
        print(vers)
        print("  ", pattern)
        print()

Pour le grec on est obligé de passer par des options bien plus complexes.
<br>Jusqu'ici je n'ai que l'hexamètre en magasin.

In [ ]:
!git clone https://github.com/epilanthanomai/hexameter.git

In [ ]:
import sys, os
sys.path.append("/content/hexameter")

In [ ]:
from unicodedata import normalize
from scan import analyze_line

vers = "μῆνιν ἄειδε, θεὰ, Πηληϊάδεω Ἀχιλῆος"

vers_nfd = normalize("NFD", vers)

analysis = analyze_line(vers_nfd)
print(analysis)

In [ ]:
def pretty_scan(line: str):
    line_nfd = normalize("NFD", line)

    analyses = analyze_line(line_nfd)
    if not analyses:
        print("Pas de scansion trouvée pour :", line)
        return

    pattern, hemistichs = analyses[0]
    left, right = hemistichs
    print(left + " | " + right)

    raw_feet = pattern.split("|")
    feet = []
    caesure_foot_idx = None

    for i, f in enumerate(raw_feet):
        if "." in f:
            caesure_foot_idx = i
            f = f.replace(".", "")
        feet.append(f)

    def to_pretty(motif: str) -> str:
        return motif.replace("+", " — ").replace("-", "˘")

    pretty_feet = []
    for i, f in enumerate(feet):
        motif = to_pretty(f)
        if caesure_foot_idx is not None and i == caesure_foot_idx:
            pretty_feet.append("‖ " + motif)
        else:
            pretty_feet.append(motif)

    print(" | ".join(pretty_feet))
    print()

In [ ]:
pretty_scan("μῆνιν ἄειδε, θεὰ, Πηληϊάδεω Ἀχιλῆος")